In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
#  https://github.com/https://github.com/mithunraj984
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm, preprocessing

In [ ]:
df=pd.read_csv('/kaggle/input/black-friday-sale/train.csv')
df.info()
df.head(5)

We have some missing data on product category but it wasn't filled on purpose.

In [ ]:
df.describe()

In [ ]:
plt.style.use('classic')


# Purchase Distribution

In [ ]:
purchase_d=df['Purchase']
purchase_d.plot(kind='hist',bins=15, 
       edgecolor = 'black',figsize=(10,6.5),title='Purchase Distribution')

Most of the purchases are between 5000 and 10k, we have some outliers purchases above 20000

# Analysis by Gender

In [ ]:
df.head()

In [ ]:
gender_count=df[['Gender','User_ID']].groupby('Gender').count()
gender_count
plt.style.use('Solarize_Light2')
gender_count.plot(kind='pie',autopct='%1.1f%%',subplots=True,title='Males vs Females',figsize=(12,7))

# Mals vs Female by Purchases

In [ ]:
gender_pu=df[['Gender','Purchase']].groupby('Gender').sum()
gender_pu
gender_pu.plot(kind='pie',autopct='%1.1f%%',subplots=True,figsize=(15,7),title='Males vs Females by Total Purchases')


# Purchases by Marital Status

In [ ]:
status_pu=df[['Marital_Status','Purchase']].groupby('Marital_Status').sum()
status_pu.plot(kind='pie',title='Total Purchases by Marital Status',autopct='%1.1f%%',subplots=True,figsize=(12,7))


**Singles buy more than married people**

# Occupation status count:

In [ ]:
ocu_count=df[['Occupation','Purchase']].groupby('Occupation').count()
ocu_count=ocu_count.sort_values(by='Purchase',ascending=False)
ocu_count.plot(kind='bar',title='Occupation count')

# Purchases by occupation:

In [ ]:
ocu_pu=df[['Occupation','Purchase']].groupby('Occupation').sum()
ocu_pu=ocu_pu.sort_values(by='Purchase',ascending=False)
ocu_pu.plot(kind='bar',title='Total Purchases by Occupation')

# Total Purchases by city

In [ ]:
city_pu=df[['City_Category','Purchase']].groupby('City_Category').sum()
city_pu=city_pu.sort_values(by='Purchase')
city_pu.plot(kind='bar',subplots=True,title='Total Purchases by City')

**City B has the highest purchases amount**

# Purchase by Age Group

In [ ]:

age_pu=df[['Age','Purchase']].groupby('Age').sum()
age_pu.plot(kind='barh')


**The 26-35 age category has the highest purchases amount**

# Product category Analysis

In [ ]:
prod_pu1=df[['Product_Category_1','Purchase']].groupby('Product_Category_1').sum()
prod_pu1=prod_pu1.sort_values(by='Purchase')
prod_pu1.plot(kind='barh',title='Product Category 1 by purchases')


In [ ]:

prod_pu2=df[['Product_Category_2','Purchase']].groupby('Product_Category_2').sum()
prod_pu2=prod_pu1.sort_values(by='Purchase')
prod_pu2.plot(kind='barh',title='Product Category 2 by purchases')


In [ ]:
prod_pu3=df[['Product_Category_3','Purchase']].groupby('Product_Category_3').sum()
prod_pu3=prod_pu1.sort_values(by='Purchase')
prod_pu3.plot(kind='barh',title='Product Category 3 by purchases')


# Top 5 products by Purchases

In [ ]:
top_prod=df[['Product_ID','Purchase']].groupby('Product_ID').sum()
top_prod=top_prod.sort_values(by='Purchase',ascending=False)
top_prod=top_prod.head(5)
top_prod.plot(kind='pie',title='Top 5 products by Purchases',subplots=True,autopct='%1.1f%%',legend=False,ylabel='',figsize=(12,7))

# Customers purchases by their years in the city

In [ ]:
city_dur=df[['Stay_In_Current_City_Years','Purchase']].groupby('Stay_In_Current_City_Years').sum()
city_dur=city_dur.sort_values(by='Purchase')
city_dur.plot(kind='bar',xlabel='Years',ylabel='Years',title='Customers purchases by their years in the city')

**It's seems that the longer customers stay in the city, the more their purchases will decline. Maybe they discover other markets...**

# Data correlation:

In [ ]:
df2=df.copy()


**Data encoding:**

In [ ]:
df2['Gender']=pd.factorize(df2.Gender)[0]
df2['Age']=pd.factorize(df2.Age)[0]
df2['City_Category']=pd.factorize(df2.City_Category)[0]
df2['Stay_In_Current_City_Years']=pd.factorize(df2.Stay_In_Current_City_Years)[0]
df2['Product_ID']=pd.factorize(df2.Product_ID)[0]
df2['User_ID']=pd.factorize(df2.User_ID)[0]




In [ ]:
df2['Product_Category_1']=pd.factorize(df2.Product_Category_1)[0]
df2['Product_Category_2']=pd.factorize(df2.Product_Category_2)[0]
df2['Product_Category_3']=pd.factorize(df2.Product_Category_3)[0]

# Data correlation:

In [ ]:
df2.corr()

In [ ]:

sns.heatmap(df2.corr(), annot = True, cmap= 'YlGnBu', fmt= '.2f')

**We can see that Age, Gender and Occcupation has a low impact on the Purchase amount, while product category and Product ID has a high impact on the purchases amount**

# Building a model to preduct Purchases amount:

**Multipe Regression Prediction:**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score

**Setting Features and terget**

In [ ]:
X= df2.values[:,0:11]
Y= df2.values[:, 11]

In [ ]:
X= preprocessing.scale(X)

**Splitting the dataset**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 100)


**Model training:**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
rfr = RandomForestRegressor()

In [ ]:
rfr.fit(X_train,y_train)

**Model accuracy:**

In [ ]:
score = rfr.score(X_train,y_train)
score

**Testing accuracy:**

In [ ]:
y_pred5 = rfr.predict(X_test)
rscore=r2_score(y_test, y_pred5)
rscore

In [ ]:
df2['Prediction']=rfr.predict(X)


**Prediction vs Actual purchase:**

In [ ]:
df2.head(10)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, linewidth = '1', label="original")
plt.plot(x_ax, y_pred5, linewidth = '1', label="predicted")
plt.title("Test data VS Predicted data")
plt.xlabel('X')
plt.ylabel('Y')
plt.legend(loc='lower right',fancybox=True, shadow=True)
plt.show() 

**Now we have a model that has been tested with %69 Accuracy that will predict the purchase amount based on the data features. Don't forget to upvote this notebook if you like it!**